In [ ]:
# Install dependencies

%pip install loguru==0.7.3 python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.6.0 torchvision==0.21.0 tqdm==4.67.1 typer==0.15.1 matplotlib==3.10.0 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 ultralytics==8.3.107 ray==2.43.0 albumentations==2.0.5 shortuuid==1.0.13

In [ ]:
# Imports

from pathlib import Path
from PIL import Image
from torch.optim.lr_scheduler import LRScheduler
from ultralytics import YOLO, settings
from ultralytics.data.dataset import YOLODataset
from ultralytics.models.yolo.detect import DetectionTrainer, DetectionValidator
from ultralytics.nn.tasks import DetectionModel
from ultralytics.utils import colorstr, LOGGER, SettingsManager
from ultralytics.utils.torch_utils import one_cycle
import gc
import glob
import io
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import shortuuid
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as vutils
import wandb
import yaml

In [ ]:
# Utils

def load_config(config_file: str) -> dict:
    """Load and return configuration from YAML file."""
    with open(config_file, "r") as f:
        return yaml.safe_load(f)

def _get_wandb_key_colab() -> str:
    try:
        from google.colab import userdata # type: ignore

        if userdata.get("WANDB_API_KEY") is not None:
            return userdata.get("WANDB_API_KEY")
        else:
            raise ValueError("No WANDB key found")
    except:
        return None

def _get_wandb_env(path: Path) -> str:
    try:
        from dotenv import dotenv_values # type: ignore

        """Get W&B API key from Colab userdata or environment variable"""

        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Could not find .env file at {path}")

        print(f"Loading secrets from {path}")

        secrets = dotenv_values(path)
        print(f"Found keys: {list(secrets.keys())}")

        if "WANDB_API_KEY" not in secrets:
            raise KeyError(f"WANDB_API_KEY not found in {path}. Available keys: {list(secrets.keys())}")

        return secrets['WANDB_API_KEY']
    except:
        return None

def get_wandb_key(path: Path = "../.env") -> str:
    return _get_wandb_key_colab() if _get_wandb_key_colab() is not None else _get_wandb_env(path)

def clear_cache():
    # Clear CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Clear Python garbage collector
    gc.collect()

def get_device() -> str:
    try:
        return 0 if torch.cuda.is_available() else "cpu"
    except Exception as e:
        print(f"Error setting device: {e}")

def remove_models():
    pt_files = glob.glob("*.pt")
    print("Files to be removed:", pt_files)

    for file in pt_files:
        os.remove(file)

In [ ]:
# Run this only once to set up the dataset cache the first time.

# Downloads the VisDrone dataset and runs the first epoch to create the label.cache file with the original labels.
# The custom dataset is later created on the fly by mapping the original labels to the new ones.
# Without this step, the custom dataset would not be created correctly.

os.environ['WANDB_MODE'] = 'disabled'

model = YOLO("yolo12n.pt")
results = model.train(
    data="VisDrone.yaml",
    device=get_device(),
    batch=16,
    workers=8,
    imgsz=640,
    epochs=1,
    exist_ok=True,
    save=False,
)

# Remove the saved data
if results.save_dir.exists():
    print(f"Removing {results.save_dir}")
    shutil.rmtree(results.save_dir)
            
del model

clear_cache()

# Remove the dummy model
remove_models()

os.environ.pop("WANDB_MODE", None)